In [45]:
from __future__ import absolute_import, division, print_function
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
from collections import defaultdict

In [46]:
data_df = pd.read_csv('test_df.csv')

In [47]:
data_df['emotion_prob'] = data_df['emotion_prob'].replace(np.nan, 0)

In [48]:
data_df = data_df[data_df.emotion_prob > 0.5]

In [49]:
data_df.reset_index(drop = True, inplace = True)

In [50]:
data_df['minute'] = data_df['time'].apply(lambda x: int(x/60) + 1)

In [51]:
data_for_heatmap = data_df.groupby(['minute', 'emotion']).count()

In [52]:
emotions = ['happy', 'neutral', 'surprise', 'sad', 'angry', 'fear'] 

In [53]:
def emotions_data_for_heatmap(emotions_list, length_of_video, data):
    emotions_data = defaultdict(lambda: [])
    for emo in emotions_list:
        for i in range(len(length_of_video)):
            try:
                emotions_data[emo].append(data.loc[i+1]['username']['{}'.format(emo)])
            except:
                pass
                emotions_data[emo].append(0)
    return emotions_data

In [54]:
emotions_data = emotions_data_for_heatmap(emotions, range(len(list(np.unique(data_df['minute'])))), data_for_heatmap)

In [55]:
trace = go.Heatmap(
    z = [
        emotions_data['happy'],
        emotions_data['neutral'],
        emotions_data['surprise'],
        emotions_data['sad'],
        emotions_data['angry'],
        emotions_data['fear']
    ],
    x = [i for i in list(np.unique(data_df['minute']))],
    y = emotions,
    colorbar = dict(
        title = 'Infrequent - Frequent',
        titleside = 'right',
        tickvals = []
    )
)

layout = go.Layout(
    title = 'Frequency of Emotions Displayed During the Class by Minute',
    xaxis = dict(
        title = 'Time (minutes)'
    ),
    yaxis = dict(
        title = 'Emotions'
    )
)

data = [trace]

fig = go.Figure(data = data, layout = layout)
offline.init_notebook_mode(connected=True)
offline.iplot(fig)